Reproducing Results

Reproduced results from code: https://github.com/mpc001/Lipreading_using_Temporal_Convolutional_Networks.git

Steps: (Majority took place through shell on personal device)

1. Cloned repository to local machine using:
    git clone --recursive https://github.com/mpc001/Lipreading_using_Temporal_Convolutional_Networks.git

2. Installed requirements using:
    pip install -r requirements.txt

3. Downloaded pre-computed Landmarks as provided by instructions from [GoogleDrive](https://bit.ly/3huI1P5)

4. Pre-Processed Mouth ROIs using crop_mouth_from_video.py as provided. To complete used the following command:

    python3 /media/taylorpap/1TBM2/DatasetML/Lipreading_using_Temporal_Convolutional_Networks-master/preprocessing/crop_mouth_from_video.py --video-direc /media/taylorpap/1TBM2/DatasetML/lipread_mp4 \
                                --landmark-direc /media/taylorpap/1TBM2/DatasetML/Lipreading_using_Temporal_Convolutional_Networks-master/landmarks/LRW_landmarks \
                                --save-direc /media/taylorpap/1TBM2/DatasetML/Lipreading_using_Temporal_Convolutional_Networks-master/datasets/visual_data \
                                --convert-gray
    
    This pointed to the file to run, the location of the dataset (Lip Reading in the Wild), the path to the landmarks downloaded earlier, and the folder in the repository to save the outputs. This script would take the video files, detect the speaking face, rotate the image so that it is facing directly up/down, and zoom/center on the mouth. It also changes the video to grey instead of color. The output is then saved as an npz file.

    This step took several hours to complete (about 20 hours).

5.  Downloaded pre-trained model from modelzoo. I used resnet18_mstcn

6. The next step according to the repository was to begin training the model using:

    CUDA_VISIBLE_DEVICES=0 python main.py --config-path <MODEL-JSON-PATH> \
                                      --annonation-direc <ANNONATION-DIRECTORY> \
                                      --data-dir <MOUTH-ROIS-DIRECTORY>

    When attempting this step, I encountered an OS error regarding saving the training logs. My guess as to the reason why was the ':' character in the datetime for creating the folder was an invalid character. To resolve, I went into their utils.py folder and changed one of the functions:

In [ ]:
def get_save_folder( args):
    # create save and log folder
    save_path = '{}/{}'.format( args.logging_dir, args.training_mode )
    save_path += '/' + datetime.datetime.now().isoformat().split('.')[0]

    #Taylor addition: Editing save path as gives an OS error
    save_path = save_path.replace(':','.')
    
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    return save_path

Adding in the replace method resolved the error. The next error when attempting to train the model was Pytorch being incompatibile with my GPU (RTX 3090). This was reolved by downloading a more recent and CUDA version of Pytorch. With that, I was able to begin training. 

7. Training took a total of approximately 45 hours. It was stopped a couple of times and resumed from checkpoint in order to use windows on computer (as code was being run through Linux).
    resuming training was done using:
    
        python3 main.py --config-path /configs/lrw_resnet18_mstcn.json \
                                      --annonation-direc /DatasetML/lipread_mp4 \
                                      --data-dir ./datasets/visual_data \
    --model-path <path to current model> \
    --init-epoch 1

The --model-path took the path to the last checkpoint to resume training. The model completed 80 epochs of training.

8. After completing training, the completed model was tested using the test set of the mouth ROI data. 

    python3 main.py --config-path /configs/lrw_resnet18_mstcn.json \
                                      --model-path /train_logs/tcn/2022-03-06T22.04.03/ckpt.best.pth.tar \
                                      --data-dir /datasets/visual_data \
                                      --test
    
Result:
'''

Model and log being saved in: ./train_logs/tcn/2022-03-07T16.10.39
2-norm of the neural network: 48.1286
Partition train loaded
Partition val loaded
Partition test loaded
Model has been successfully loaded from /media/taylorpap/1TBM2/DatasetML/Lipreading_using_Temporal_Convolutional_Networks-master/train_logs/tcn/2022-03-06T22.04.03/ckpt.best.pth.tar
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 782/782 [00:39<00:00, 19.92it/s]
25000 in total	CR: 0.86744
Test-time performance on partition test: Loss: 0.5086	Acc:0.867

'''
The result was a score of nearly 87% accuracy, above the expected accuracy score:


|       Architecture      |   Acc.   | FLOPs (G) | url | size (MB)|

resnet18_mstcn                 |   85.5   |    10.31  |[GoogleDrive](https://bit.ly/3glF4k5) or [BaiduDrive](https://bit.ly/3513Ror) (key: um1q) |139|



Conclusion:

The code provided did not run perfectly "out of the box" and required going into the scripts provided in order to resolve a simple issue. However, the results were impressive and training the model (for me) was likely completed in less time than for others due to my hardware. The largest downside after completing the model is the inability to test it on other data. The model takes 3 inputs that are created during the preprocessing step in the packaged file, meaning raw video data is unable to be used to see how the model performs. I am sure this would be a lower real world perfomance score than what was achieved. The preprocessing script is also tailored to be used for the lipreading in the wild dataset, so I havent been able to attempt it on unseen data (at least yet), to see how the model would perform in a less ideal scenario and to see how the preprocessing works without being provided with pre-computed landmarks for each video. 

For my project I may attempt to modify this approach to preprocessing to make it more applicable to other video files, and to see how well my own model will do when using it on complete sentences/real world data.